# SQL querying

In [1]:
from main.database_utils import DatabaseConnector
from sqlalchemy import text, create_engine, MetaData, Table

In [2]:
dc = DatabaseConnector()
dc.creds = dc.read_db_creds(dc.target_creds_path)
# Create engine
dc.engine = create_engine(
    f"postgresql://{dc.creds['RDS_USER']}:{dc.creds['RDS_PASSWORD']}@{dc.creds['RDS_HOST']}:{dc.creds['RDS_PORT']}/{dc.creds['RDS_DATABASE']}"
)

In [3]:
from sqlalchemy import text
# define a SQL execution function
def execute_SQL(connection, query):
    """Executes the given SQL query using the provided connection."""
    return connection.execute(query)

### Task 1. How many stores does each country has?
The Operations team would like to know which countries we currently operate in and which country now has the most stores. Perform a query on the database to get the information, it should return the following information:

+----------+-----------------+
| country  | total_no_stores |
+----------+-----------------+
| GB       |             265 |
| DE       |             141 |
| US       |              34 |
+----------+-----------------+
Note: DE is short for Deutschland(Germany)

In [4]:
# Define the SQL query
query = text("""
    SELECT
        country_code AS country,
        COUNT(*) AS total_no_stores
    FROM
        dim_store_details
    GROUP BY
        country_code
    ORDER BY
        total_no_stores DESC;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    stores_count = result.fetchall()

    # Print results
    for row in stores_count:
        print(f"Country: {row[0]}, Total Stores: {row[1]}")  # Access by index


Country: GB, Total Stores: 266
Country: DE, Total Stores: 141
Country: US, Total Stores: 34


### Task 2. which locations currently have most stores?

The business stakeholders would like to know which locations currently have the most stores.

They would like to close some stores before opening more in other locations.

Find out which locations have the most stores currently. The query should return the following:
```
+-------------------+-----------------+
|     locality      | total_no_stores |
+-------------------+-----------------+
| Chapletown        |              14 |
| Belper            |              13 |
| Bushley           |              12 |
| Exeter            |              11 |
| High Wycombe      |              10 |
| Arbroath          |              10 |
| Rutherglen        |              10 |
+-------------------+-----------------+
```

In [5]:
# Define the SQL query
query = text("""
    SELECT
        locality,
        COUNT(*) AS total_no_stores
    FROM
        dim_store_details
    GROUP BY
        locality
    ORDER BY
        total_no_stores DESC
    LIMIT 7;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    locations_count = result.fetchall()

    # Print results
    for row in locations_count:
        print(f"Locality: {row[0]}, Total Stores: {row[1]}")  # Access by index

Locality: Chapletown, Total Stores: 14
Locality: Belper, Total Stores: 13
Locality: Bushey, Total Stores: 12
Locality: Exeter, Total Stores: 11
Locality: High Wycombe, Total Stores: 10
Locality: Arbroath, Total Stores: 10
Locality: Rutherglen, Total Stores: 10


### Task 3. which months produce largest amount of sales?

Query the database to find out which months have produced the most sales. The query should return the following information:
```
+-------------+-------+
| total_sales | month |
+-------------+-------+
|   673295.68 |     8 |
|   668041.45 |     1 |
|   657335.84 |    10 |
|   650321.43 |     5 |
|   645741.70 |     7 |
|   645463.00 |     3 |
+-------------+-------+
```

In [6]:
# Define the SQL query
query = text("""
    SELECT 
    SUM(CAST(dp.product_price AS NUMERIC) * ot.product_quantity) AS total_sales,
    dt.month
    FROM 
        orders_table ot
    JOIN 
        dim_products dp ON ot.product_code = dp.product_code
    JOIN 
        dim_date_times dt ON ot.date_uuid = dt.date_uuid
    GROUP BY 
        dt.month
    ORDER BY 
        total_sales DESC
    LIMIT 6;

""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    sales_count = result.fetchall()

    # Print results
    for row in sales_count:
        print(f"Total Sales: {row[0]:.2f}, Month: {row[1]}")  # Access by index and format total_sales

Total Sales: 673295.68, Month: 8
Total Sales: 668041.45, Month: 1
Total Sales: 657335.84, Month: 10
Total Sales: 650321.43, Month: 5
Total Sales: 645741.70, Month: 7
Total Sales: 645463.00, Month: 3


### Task 4. how much sales from online and offline?

The company is looking to increase its online sales.

They want to know how many sales are happening online vs offline.

Calculate how many products were sold and the amount of sales made for online and offline purchases.

You should get the following information:
```
+------------------+-------------------------+----------+
| numbers_of_sales | product_quantity_count  | location |
+------------------+-------------------------+----------+
|            26957 |                  107739 | Web      |
|            93166 |                  374047 | Offline  |
+------------------+-------------------------+----------+
```

In [7]:
# Define the SQL query
query = text("""
SELECT 
    COUNT(*) AS numbers_of_sales, 
    SUM(ot.product_quantity) AS product_quantity_count, 
    CASE 
        WHEN ds.store_code LIKE 'WEB%' THEN 'Web' 
        ELSE 'Offline' 
    END AS location
FROM 
    orders_table ot
JOIN 
    dim_store_details ds ON ot.store_code = ds.store_code
GROUP BY 
    location;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    sales_count = result.fetchall()

    # Print results
    for row in sales_count:
        print(f"numbers_of_sales: {row[0]:.2f}, product_quantity_count: {row[1]}, location: {row[2]}")  # Access by index and format total_sales

numbers_of_sales: 26957.00, product_quantity_count: 107739, location: Web
numbers_of_sales: 93166.00, product_quantity_count: 374047, location: Offline


### Task 5. what percentage of sales come from each type of stores?

The sales team wants to know which of the different store types is generated the most revenue so they know where to focus.

Find out the total and percentage of sales coming from each of the different store types.

The query should return:
```
+-------------+-------------+---------------------+
| store_type  | total_sales | percentage_total(%) |
+-------------+-------------+---------------------+
| Local       |  3440896.52 |               44.87 |
| Web portal  |  1726547.05 |               22.44 |
| Super Store |  1224293.65 |               15.63 |
| Mall Kiosk  |   698791.61 |                8.96 |
| Outlet      |   631804.81 |                8.10 |
+-------------+-------------+---------------------+
```

In [8]:
# Define the SQL query
query = text("""
SELECT 
    ds.store_type,
    SUM(CAST(dp.product_price AS NUMERIC) * ot.product_quantity) AS total_sales,
    ROUND(
        (SUM(CAST(dp.product_price AS NUMERIC) * ot.product_quantity) / 
        (SELECT SUM(CAST(dp.product_price AS NUMERIC) * ot.product_quantity)
         FROM orders_table ot
         JOIN dim_products dp ON ot.product_code = dp.product_code)) * 100, 2
    ) AS percentage_total
FROM 
    orders_table ot
JOIN 
    dim_products dp ON ot.product_code = dp.product_code
JOIN 
    dim_store_details ds ON ot.store_code = ds.store_code
GROUP BY 
    ds.store_type
ORDER BY 
    total_sales DESC;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    sales_percentage = result.fetchall()

    # Print results
    for row in sales_percentage:
        print(f"store_type: {row[0]}, total_sales: {row[1]}, percentage_total(%){row[2]}")  # Access by index and format total_sales

store_type: Local, total_sales: 3440896.52, percentage_total(%)44.56
store_type: Web Portal, total_sales: 1726547.05, percentage_total(%)22.36
store_type: Super Store, total_sales: 1224293.65, percentage_total(%)15.85
store_type: Mall Kiosk, total_sales: 698791.61, percentage_total(%)9.05
store_type: Outlet, total_sales: 631804.81, percentage_total(%)8.18


### Task 6. Which month in each year produce biggest cost sales?

The company stakeholders want assurances that the company has been doing well recently.

Find which months in which years have had the most sales historically.

The query should return the following information:
```
+-------------+------+-------+
| total_sales | year | month |
+-------------+------+-------+
|    27936.77 | 1994 |     3 |
|    27356.14 | 2019 |     1 |
|    27091.67 | 2009 |     8 |
|    26679.98 | 1997 |    11 |
|    26310.97 | 2018 |    12 |
|    26277.72 | 2019 |     8 |
|    26236.67 | 2017 |     9 |
|    25798.12 | 2010 |     5 |
|    25648.29 | 1996 |     8 |
|    25614.54 | 2000 |     1 |
+-------------+------+-------+
```

In [9]:
# Define the SQL query
query = text("""
SELECT 
    SUM(CAST(dp.product_price AS NUMERIC) * ot.product_quantity) AS total_sales,
    dt.year,
    dt.month
FROM 
    orders_table ot
JOIN 
    dim_products dp ON ot.product_code = dp.product_code
JOIN 
    dim_date_times dt ON ot.date_uuid = dt.date_uuid
GROUP BY 
    dt.year, dt.month
ORDER BY 
    total_sales DESC
LIMIT 10;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    total_sales = result.fetchall()

    # Print results
    for row in total_sales:
        print(f"total_sales: {row[0]}, year: {row[1]}, month: {row[2]}")  # Access by index and format total_sales

total_sales: 27936.77, year: 1994, month: 3
total_sales: 27356.14, year: 2019, month: 1
total_sales: 27091.67, year: 2009, month: 8
total_sales: 26679.98, year: 1997, month: 11
total_sales: 26310.97, year: 2018, month: 12
total_sales: 26277.72, year: 2019, month: 8
total_sales: 26236.67, year: 2017, month: 9
total_sales: 25798.12, year: 2010, month: 5
total_sales: 25648.29, year: 1996, month: 8
total_sales: 25614.54, year: 2000, month: 1


### Task 7. Staff headcount?

The operations team would like to know the overall staff numbers in each location around the world. Perform a query to determine the staff numbers in each of the countries the company sells in.

The query should return the values:
```
+---------------------+--------------+
| total_staff_numbers | country_code |
+---------------------+--------------+
|               13307 | GB           |
|                6123 | DE           |
|                1384 | US           |
+---------------------+--------------+
```

In [10]:
# Define the SQL query
query = text("""
SELECT 
    country_code,
    SUM(staff_numbers) AS total_staff
FROM 
    dim_store_details
GROUP BY 
    country_code
ORDER BY 
    total_staff DESC;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    total_staff = result.fetchall()

    # Print results
    for row in total_staff:
        print(f"total_staff_numbers: {row[0]}, country_code: {row[1]}")  # Access by index and format total_sales


total_staff_numbers: GB, country_code: 13307
total_staff_numbers: DE, country_code: 6123
total_staff_numbers: US, country_code: 1384


### Task 8. which German store type selling the most?

The sales team is looking to expand their territory in Germany. Determine which type of store is generating the most sales in Germany.

The query will return:
```
+--------------+-------------+--------------+
| total_sales  | store_type  | country_code |
+--------------+-------------+--------------+
|   198373.57  | Outlet      | DE           |
|   247634.20  | Mall Kiosk  | DE           |
|   384625.03  | Super Store | DE           |
|  1109909.59  | Local       | DE           |
+--------------+-------------+--------------+
```

In [11]:
# Define the SQL query
query = text("""
SELECT 
    SUM(CAST(dp.product_price AS NUMERIC) * ot.product_quantity) AS total_sales,
    ds.store_type,
    ds.country_code
FROM 
    orders_table ot
JOIN 
    dim_products dp ON ot.product_code = dp.product_code
JOIN 
    dim_store_details ds ON ot.store_code = ds.store_code
WHERE 
    ds.country_code = 'DE'
GROUP BY 
    ds.store_type, ds.country_code
ORDER BY 
    total_sales ASC;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    total_staff = result.fetchall()

    # Print results
    for row in total_staff:
        print(f"total_sales: {row[0]}, store_type: {row[1]}, country_code: {row[2]}")  # Access by index and format total_sales

total_sales: 198373.57, store_type: Outlet, country_code: DE
total_sales: 247634.20, store_type: Mall Kiosk, country_code: DE
total_sales: 384625.03, store_type: Super Store, country_code: DE
total_sales: 1109909.59, store_type: Local, country_code: DE


### Task 9. How quickly the company make sales?

Sales would like the get an accurate metric for how quickly the company is making sales.

Determine the average time taken between each sale grouped by year, the query should return the following information:
```
 +------+-------------------------------------------------------+
 | year |                           actual_time_taken           |
 +------+-------------------------------------------------------+
 | 2013 | "hours": 2, "minutes": 17, "seconds": 12, "millise... |
 | 1993 | "hours": 2, "minutes": 15, "seconds": 35, "millise... |
 | 2002 | "hours": 2, "minutes": 13, "seconds": 50, "millise... | 
 | 2022 | "hours": 2, "minutes": 13, "seconds": 6,  "millise... |
 | 2008 | "hours": 2, "minutes": 13, "seconds": 2,  "millise... |
 +------+-------------------------------------------------------+
 ```

In [13]:
# Define the SQL query
query = text("""
WITH sales_with_times AS (
    SELECT 
        dt.year::int AS year,
        TO_TIMESTAMP(dt.year || '-' || dt.month || '-' || dt.day || ' ' || dt.timestamp, 'YYYY-MM-DD HH24:MI:SS') AS sale_timestamp
    FROM orders_table o
    JOIN dim_date_times dt ON o.date_uuid = dt.date_uuid
),
sales_with_diff AS (
    SELECT 
        year,
        sale_timestamp,
        LEAD(sale_timestamp) OVER (PARTITION BY year ORDER BY sale_timestamp) AS next_sale_timestamp
    FROM sales_with_times
),
time_differences AS (
    SELECT 
        year,
        sale_timestamp,
        next_sale_timestamp,
        next_sale_timestamp - sale_timestamp AS time_diff
    FROM sales_with_diff
    WHERE next_sale_timestamp IS NOT NULL
),
avg_time_per_year AS (
    SELECT 
        year,
        AVG(time_diff) AS avg_time_diff
    FROM time_differences
    GROUP BY year
)
SELECT 
    year,
    TO_CHAR(avg_time_diff, 'HH24 "hours", MI "minutes", SS "seconds", MS "milliseconds"') AS actual_time_taken
FROM avg_time_per_year
ORDER BY avg_time_diff DESC
limit 5;
""")

# Use the connection from the engine
with dc.engine.connect() as connection:
    # Fetch all results
    result = execute_SQL(connection, query)
    time_taken = result.fetchall()

    # Print results
    for row in time_taken:
        print(f"year: {row[0]}, actual_time_taken: {row[1]}") 

year: 2013, actual_time_taken: 02 hours, 17 minutes, 13 seconds, 712 milliseconds
year: 1993, actual_time_taken: 02 hours, 15 minutes, 35 seconds, 481 milliseconds
year: 2002, actual_time_taken: 02 hours, 13 minutes, 39 seconds, 915 milliseconds
year: 2008, actual_time_taken: 02 hours, 13 minutes, 03 seconds, 770 milliseconds
year: 2022, actual_time_taken: 02 hours, 13 minutes, 02 seconds, 003 milliseconds
